In [1]:
%load_ext autoreload
%autoreload 2
import sys 
import plotly.express as px
import plotly.graph_objects as go
import re
import time

import utils
from slurm import SlurmClient

# Slurm Queue

In [4]:
!sinfo

PARTITION AVAIL  TIMELIMIT  NODES  STATE NODELIST
cpu          up 7-00:00:00      2   idle cpu-c7i12xlarge-[0-1]
q1*          up 7-00:00:00      1   comp h100-st-p548xlarge-231
q1*          up 7-00:00:00    306  alloc h100-st-p548xlarge-[4-11,13-25,27-51,53-55,58-97,99-135,139-143,145-149,162,165-188,195-197,199-215,217-226,235-239,246,249-250,268-269,279-284,286-292,296,298-353,355,376-380,392-403,410-422,431-434]
q1*          up 7-00:00:00     44   plnd h100-st-p548xlarge-[26,52,136-138,193-194,244,271-278,293-295,357-358,360-375,404-405,408,424-425,435-436]
q1*          up 7-00:00:00      2  idle* h100-st-p548xlarge-[241,243]
q1*          up 7-00:00:00      3  down* h100-st-p548xlarge-[0,164,359]
q1*          up 7-00:00:00     13  drain h100-st-p548xlarge-[150-161,354]
q1*          up 7-00:00:00     26    mix h100-st-p548xlarge-[1,3,144,163,189-192,228,245,252-264,266-267,356]


In [35]:
sl = SlurmClient()
q = sl.get_queue_summary()

print("Total:", q.NODES.sum())
q

Total: 468


,ACCOUNT,ST,NODES
0,ar-ai-hipri,CG,4
1,ar-ai-hipri,PD,96
2,ar-ai-hipri,R,330
3,ar-ai-hipri,RH,1
4,ar-ai-midpri,R,16
5,ar-ai-research-interns,R,3
6,ar-ai-voice-hipri,R,18


In [43]:
sl.get_recent_queue('all')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
22,9476_4,all,sft-eval,jiuhai,R,2024-08-12T19:22:15,0:43,16,43


In [42]:
sl.get_recent_queue('ar-ai-research-interns')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
27,9294,ar-ai-research-interns,jupyter,imzyc,R,2024-08-10T08:11:10,2-11:08:28,1,212908
28,8363,ar-ai-research-interns,bash,yfzhuang,R,2024-08-09T18:21:10,3-00:58:28,1,262708
29,8364,ar-ai-research-interns,bash,yfzhuang,R,2024-08-09T18:21:10,3-00:58:28,1,262708


In [6]:
sl.get_recent_queue('ar-ai-voice-hipri')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
10,7143,ar-ai-voice-hipri,bash,arashe,R,2024-08-05T23:03:47,6-15:46:30,1,575190
11,7123,ar-ai-voice-hipri,bash,arashe,R,2024-08-05T19:25:54,6-19:24:23,1,588263


In [54]:
q = sl.get_queue_summary()
running = {}
for _, row in q.iterrows():
    if row.ST == 'R':
        running[row.ACCOUNT] = row.NODES
print(running)

quota = {'ar-ai-research-interns': 8,
         'ar-ai-voice-hipri': 50, 'ar-ai-hipri': 400}

evict_ids = {}
for acc in running:
    if acc not in quota:  # quota = 0
        print(acc)

{'all': 17, 'ar-ai-hipri': 274, 'ar-ai-midpri': 2, 'ar-ai-research-interns': 3, 'ar-ai-voice-hipri': 53}
all
ar-ai-midpri


In [30]:
q_summary = q.groupby(['ACCOUNT', 'ST']).aggregate(
    {'NODES': 'sum'}).reset_index()
q_summary

,ACCOUNT,ST,NODES
0,all,R,17
1,ar-ai-hipri,R,280
2,ar-ai-midpri,R,2
3,ar-ai-research-interns,R,3
4,ar-ai-voice-hipri,PD,94
5,ar-ai-voice-hipri,R,53


# Pretrain

In [48]:
MM9_CONF_DIR = "/fsx_0/user/tranx/experiments/llm_mm_aligner/stage1_mm9"
envvar = {
    "NUM": 7,
    "JSON_CONFIG": f"fbl_pretrain_MM9_70B_Llama31_336px_2nodes.json",
    # "CONDA_ENV": "aligner_v7",
}

sbatch_base = "/fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh"
sbatch_overwrite = {
    "job-name": "test",
    "nodes": 2,
    "ntasks": 2
}

MM9_CONF_DIR = "/fsx_0/user/tranx/experiments/llm_mm_aligner/stage1_mm9"
JSON_CONFIG = f"{MM9_CONF_DIR}/xxxfbl_pretrain_MM9_70B_Llama31_336px_2nodes.json"
ALIGNER_PARENT_DIR = "/fsx_0/user/tranx"

envvar_string = f"{JSON_CONFIG} {ALIGNER_PARENT_DIR}"

sbatch_vars_string = []
for k, v in sbatch_overwrite.items():
    sbatch_vars_string.append(f"--{k}={v}")
sbatch_vars_string = ' '.join(sbatch_vars_string)

# cmd = f'sbatch --parsable {sbatch_vars_string} {sbatch_base} {envvar_string}'
cmd = f'sbatch --parsable {sbatch_vars_string} {sbatch_base} a b'
print(cmd)

job_id = utils.get_bash_output(cmd, print_output=True)
job_id = int(job_id)

sbatch --parsable --job-name=test --nodes=2 --ntasks=2 /fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh a b
9659



In [29]:
!tail -f /fsx_0/user/tranx/output/slurm_logs/output_9402.txt

/var/spool/slurmd/job09402/slurm_script: line 23: conda: command not found
/var/spool/slurmd/job09402/slurm_script: line 24: conda: command not found
Using conda environment:
Error: CONDA_DEFAULT_ENV is not set to aligner_v7
^C


# Stage 2

# Evals